In [1]:
# # Loading data and fitting a choice model

# The package expects your data to live in a single .mat file which should contain a struct called `rawdata`. Each element of `rawdata` should have data for one behavioral trial and `rawdata` should contain the following fields with the specified structure:

# - `rawdata.leftbups`: row-vector containing the relative timing, in seconds, of left clicks on an individual trial. 0 seconds is the start of the click stimulus.
# - `rawdata.rightbups`: row-vector containing the relative timing in seconds (origin at 0 sec) of right clicks on an individual trial. 0 seconds is the start of the click stimulus.
# - `rawdata.T`: the duration of the trial, in seconds. The beginning of a trial is defined as the start of the click stimulus. The end of a trial is defined based on the behavioral event “cpoke_end”. This was the Hanks convention.
# - `rawdata.pokedR`: `Bool` representing the animal choice (1 = right).

# ## Fitting the model

# Once your data is correctly formatted and you have the package added in julia, you are ready to fit the model. You need to write a slurm script to use spock's resources and a .jl file to load the data and fit the model. See examples of each below. These files are also located in the package in the `examples` directory.

# ### Example slurm script

# This will start a job called `fit_choice_model`. Output will be written to a log file called `fit_choice_model.out`. This will run on the `Brody` partition of spock for 12 hours, using 44 cores and 64 GB of memory. You'll notice that we load the julia module (like we did when we added the package) and then we call julia (`-p 44` uses the 44 cores) and ask it to run the .jl file.

# ```
# #!/usr/bin/env bash
#
# #SBATCH -J 'fit_choice_model'
# #SBATCH -o ../logs/fit_choice_model.out
# #SBATCH -p Brody
# #SBATCH --time=12:00:00
# #SBATCH --mem=64000
# #SBATCH -c 44
#
# module load julia/1.0.0
# julia -p 44 ./fit_choice_model.jl
# ```

In [2]:
# ### Example .jl file
# Blah blah blah

using PulseInputDDM, Flatten

In [3]:
# ### Load some data
# Blah blah blah

data = load_choice_data("../choice model/example_matfile.mat");

In [4]:
# ### Set options for optimization
# Blah blah blah

n = 53

options = choiceoptions(fit = vcat(trues(9)),
    lb = vcat([0., 8., -5., 0., 0., 0.01, 0.005], [-30, 0.]),
    ub = vcat([2., 30., 5., 100., 2.5, 1.2, 1.], [30, 1.]))

choiceoptions
  fit: Array{Bool}((9,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1]
  lb: Array{Float64}((9,)) [0.0, 8.0, -5.0, 0.0, 0.0, 0.01, 0.005, -30.0, 0.0]
  ub: Array{Float64}((9,)) [2.0, 30.0, 5.0, 100.0, 2.5, 1.2, 1.0, 30.0, 1.0]


In [8]:
x0 = vcat([0.1, 15., -0.1, 20., 0.5, 0.8, 0.008], [0.,0.01])

9-element Vector{Float64}:
  0.1
 15.0
 -0.1
 20.0
  0.5
  0.8
  0.008
  0.0
  0.01

In [6]:
# ### Load some data
# Blah blah blah
save_file = "../choice model/example_results.mat"

#if you've already ran the optimization once and want to restart from where you stoped, this will reload those parameters
if isfile(save_file)
    θ, options = reload_choice_model(save_file)
else
    θ = Flatten.reconstruct(θchoice(), x0)
end

(θchoice{θz{Float64}, Float64}
  θz: θz{Float64}
  bias: Float64 0.6814442212446054
  lapse: Float64 0.5003949673659975
, choiceoptions
  fit: Array{Bool}((9,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1]
  lb: Array{Float64}((9,)) [0.0, 8.0, -5.0, 0.0, 0.0, 0.01, 0.005, -30.0, 0.0]
  ub: Array{Float64}((9,)) [2.0, 30.0, 5.0, 100.0, 2.5, 1.2, 1.0, 30.0, 1.0]
)

In [10]:
model = choiceDDM(θ=θ, data=data, n=n);

In [11]:
# ### Optimize stuff
# Blah blah blah

model, = optimize(model, options; iterations=5, outer_iterations=1)

Fminbox
-------
Initial mu = 0.000239451

Fminbox iteration 1
-------------------
Calling inner optimizer with mu = 0.000239451

(numbers below include barrier contribution)
Iter     Function value   Gradient norm 
     0     1.751876e+01     9.826801e+01
 * time: 0.01704096794128418

Exiting inner optimizer with x = [0.12440028155538478, 14.999905238456675, -0.12704778468241013, 19.999452481025894, 0.6996703345624105, 1.1398211855275555, 0.020373338843095307, 0.6570122273138738, 0.47641135369147464]
Current distance to box: 0.0153733
Decreasing barrier term μ.



(choiceDDM{θchoice{θz{Float64}, Float64}, Vector{PulseInputDDM.choicedata{PulseInputDDM.choiceinputs{PulseInputDDM.clicks, PulseInputDDM.binned_clicks}}}, θprior{Float64}}
  θ: θchoice{θz{Float64}, Float64}
  data: Array{PulseInputDDM.choicedata{PulseInputDDM.choiceinputs{PulseInputDDM.clicks, PulseInputDDM.binned_clicks}}}((10,))
  n: Int64 53
  cross: Bool false
  θprior: θprior{Float64}
,  * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     1.486091e+01

 * Found with
    Algorithm:     Fminbox with BFGS

 * Convergence measures
    |x - x'|               = 8.98e-01 ≰ 1.0e-10
    |x - x'|/|x'|          = 3.58e-02 ≰ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 1.0e-09
    |g(x)|                 = 4.76e-01 ≰ 1.0e-03

 * Work counters
    Seconds run:   9  (vs limit 170000)
    Iterations:    1
    f(x) calls:    6
    ∇f(x) calls:   6
)

In [8]:
# ### Compute Hessian and the confidence interavls
# Blah blah blah

H = Hessian(model)
CI, = CIs(H);

┌ Warning: Hessian is not positive definite. Approximated by closest PSD matrix.
│             ||ϵ||/||H|| is 8.46979164769424
└ @ pulse_input_DDM /usr/people/briandd/Projects/pulse_input_DDM/src/base_model.jl:19


In [10]:
# ### Save results
# Blah blah blah

save_choice_model(save_file, model, options, CI)

In [10]:
# ### Getting help
# To get more details about how any function in this package works, in julia you can type `?` and then the name of the function. Documentation will display in the REPL.